In [154]:
import numpy as np
# import scipy as np

In [155]:
""" 
    General data structures like twists and screws
    @author Wil Louis Rothman
    November 2024 - December 2024
"""

import numpy as np

class Twist:
    """
        General twist as defined in EECS C106A.

        >>> v, omega = [3, 4, 5], [pi, pi/2, 0]
        >>> xi = Twist(v, omega)
        >>> xi
        Twist(v = [3, 4, 5], ω = [pi, pi/2, 0])
        >>> 2 * xi.as_array()
        np.array([6, 8, 10, 2pi, pi, 0])
        >>> 2 * xi.vector # same as previous
        np.array([6, 8, 10, 2pi, pi, 0])

    """
    def __init__(self, linear_velocity, angular_velocity):
        linear_velocity, angular_velocity = list(linear_velocity), list(angular_velocity)

        assert len(linear_velocity) == 3 and len(angular_velocity) == 3, \
            f"linear and angular velocities must be 3D vectors, \
                instead got {linear_velocity} (length {len(linear_velocity)}) \
                    and {angular_velocity} (length {len(angular_velocity)})"
        
        self.vector = np.array(list(linear_velocity) + list(angular_velocity))

        assert np.issubdtype(self.vector.dtype, np.number), \
            "linear and angular velocities must be representable by a \
                numerical value"
        
    def __repr__(self):
        return f"Twist(v = {self.vector[:3]}, ω = {self.vector[3:]})"

    def as_array(self):
        return self.vector
    

class Revolute(Twist):
    """
        Pure rotation as defined in EECS C106A.
        xi = [-ω x q    ω]^T
        where omega := angular velocity
        and       q := some point on rotation axis
    """
    def __init__(self, omega, q):
        super().__init__(-np.cross(omega, q), omega)

class Prismatic(Twist):
    """
        Pure translation as defined in EECS C106A.
        xi = [v     0]^T
        where v := linear velocity
    """
    def __init__(self, v):
        super().__init__(v, [0, 0, 0])

class GeneralScrew(Twist):
    """
        General screw motion as defined in EECS C106A.
        xi = [-ω x q + hω   ω]^T
        where h := is pitch
    """
    def __init__(self, omega, q, h):
        super().__init__(-np.cross(omega, q) + h * omega, omega)

In [156]:
v, omega = [3, 4, 5], [np.pi, np.pi/2, 0]
xi = Twist(v, omega)
xi

Twist(v = [3. 4. 5.], ω = [3.14159265 1.57079633 0.        ])

In [157]:
Revolute([0, np.pi / 2, 0], [1, 0, 0])

Twist(v = [-0.         -0.          1.57079633], ω = [0.         1.57079633 0.        ])

In [158]:
Prismatic([3, 4, 5])

Twist(v = [3 4 5], ω = [0 0 0])

In [159]:
GeneralScrew([0, np.pi / 2, 0], [0, 1, 0], 1)

Twist(v = [0.         1.57079633 0.        ], ω = [0.         1.57079633 0.        ])

Repulsion model

In [160]:
class Pole:
    """
        Represents the pole. We are given two (unequal) points of the pole
    """
    def __init__(self, q_pole1, q_pole2):
        q_pole1, q_pole2 = list(q_pole1), list(q_pole2)
        assert len(q_pole1) == 3 and len(q_pole2) == 3, f"q_pole1 and q_pole2 must be 3D coordinates but instead got q1: {q_pole1} and q2: {q_pole2}"
        assert q_pole1 != q_pole2, f"Cannot define pole line as q1 = q2 = {q_pole1}"
        self.q1, self.q2 = np.array(q_pole1), np.array(q_pole2)

    def dist(self, p):
        """ Outputs distance between the line representing the pole and 3D coordinate vector p """
        v = self.q2 - self.q1
        w = p - self.q1
        return np.linalg.norm(np.cross(w, v)) / np.linalg.norm(v)

In [161]:
pole = Pole([0, 0, 0], [0, 0, 1])
pole.dist([1, 1, 1])

np.float64(1.4142135623730951)

In [162]:
!pip install scipy

You should consider upgrading via the 'C:\Users\Wil\ee106a-final\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [185]:
"""
    Underlying optimization model. See `../formulas.tex` for more information.
    @author Wil Louis Rothman
    November 2024 - December 2024
"""

import numpy as np
import scipy as sp
from scipy.stats import norm

from twists import *


sample_size = 500 # the number of standard normal samples to take of the pole to calculate repulsion

class Pole:
    """
        Represents the pole. We are given two (unequal) points of the pole
    """
    def __init__(self, q_pole1, q_pole2):
        q_pole1, q_pole2 = list(q_pole1), list(q_pole2)
        assert len(q_pole1) == 3 and len(q_pole2) == 3, f"q_pole1 and q_pole2 must be 3D coordinates but instead got q1: {q_pole1} and q2: {q_pole2}"
        assert q_pole1 != q_pole2, f"Cannot define pole line as q1 = q2 = {q_pole1}"
        self.q1, self.q2 = np.array(q_pole1), np.array(q_pole2)
        self.v = self.q2 - self.q1

    def dist(self, p):
        """ Outputs distance between the line representing the pole and 3D coordinate vector p """
        w = p - self.q1
        return np.linalg.norm(np.cross(w, self.v)) / np.linalg.norm(self.v)
    
    def projection(self, p):
        """ Gets the projection (i.e. closest point) of p to the line """
        w = p - self.q1
        v_dot_v = np.dot(self.v, self.v)
        
        # Ensure no division by zero
        if np.isclose(v_dot_v, 0):
            raise ValueError("Degenerate pole: the direction vector has near-zero magnitude.")
        
        projection_factor = np.dot(w, self.v) / v_dot_v
        return self.q1 + projection_factor * self.v

    
    def get_sample_points(self, x_ef, sample_size):
        """ Get samples of points for repulsion (Monte Carlo integration) """
        self.projection(x_ef)
        samples = norm.rvs(loc=0, scale=1, size=sample_size) # Standard normal distribution
        
        p_vector = np.array([]) # the calculated sample points 
        for sample in samples:
            v_norm = np.linalg.norm(self.v)
            p_i = self.q1 + sample * v_norm
            p_vector = np.append(p_vector, p_i)
        return p_vector


class RepulsionModel:
    def __init__(self, pole, x_ef, sample_size, k):
        x_ef = list(x_ef)
        assert len(x_ef) == 3, f"len_ef must be 3 instead got {x_ef}"
        assert isinstance(pole, Pole), f"type of pole must be Pole, got {type(pole)}."

        self.pole = pole 
        self.x_ef = np.array(x_ef)
        self.sample_size = sample_size
        self.k = k

        # Get r and r_hat values
        r_vector, r_hat_vector = self.get_r_values()

        # Compute total repulsion force as a 3D vector
        self.repulsion_force = np.zeros(3)  # Initialize as vector
        for i in range(len(r_vector)):
            r_i, r_hat_i = r_vector[i], r_hat_vector[i]

            if r_i > 0:  # Avoid division by zero
                repulsion_i = (self.k / r_i**2) * r_hat_i
                self.repulsion_force += repulsion_i




    def get_r_values(self):
        """ 
            Calculate repulsion from the pole from the pole at a variety of points, 
            mostly centered around the closest point on the line to the EF

            @returns r_values, r_hat_values 
        """
        p_vector = self.pole.get_sample_points(self.x_ef, self.sample_size)

        r_vector = np.array([])  # r_i = ||x_ef - p_i||
        r_hat_vector = np.array([]) # (r_hat)_i = (x_ef - p_i) / ||x_ef - p_i||
        for p_i in p_vector:
            r_vector = np.append(r_vector, np.linalg.norm(self.x_ef - p_i))
            r_hat_vector = np.append(r_hat_vector, (self.x_ef - p_i) / np.linalg.norm(self.x_ef - p_i))

        return r_vector, r_hat_vector




In [186]:
x_ef = random_triple()
pole = Pole([0, 0, 0], [0, 0, 10])  # Pole along the z-axis
repulsion_model = RepulsionModel(pole, x_ef, sample_size=5000, k=1)
force = repulsion_model.repulsion_force
print(f"vector{tuple(x_ef)}, {tuple(repulsion_model.repulsion_force)})")


vector(np.float64(-7.382932209293616), np.float64(-2.171016031103714), np.float64(-6.6521341721898235)), (np.float64(-40.816489279105866), np.float64(-40.816489279105866), np.float64(-40.816489279105866)))


In [187]:
from random import random

def random_triple():
   return np.random.rand(3) * 20 - 10


def plot_desmos(n=20):
    for _ in range(n):
        x_ef = random_triple()
        pole = Pole([0, 0, 0], [0, 0, 10])  # Pole along the z-axis
        repulsion_model = RepulsionModel(pole, x_ef, sample_size=5000, k=1)
        print(f"vector(({', '.join(map(str, x_ef.round(2)))}), ({', '.join(map(str, repulsion_model.repulsion_force.round(2)))}))")

plot_desmos()

vector((-0.77, -9.02, -5.3), (-24.28, -24.28, -24.28))
vector((-0.02, 6.57, -1.55), (8.35, 8.35, 8.35))
vector((-8.35, 6.51, -6.42), (-5.83, -5.83, -5.83))
vector((1.21, 0.47, -9.81), (-8.67, -8.67, -8.67))
vector((7.1, 1.89, -2.17), (13.0, 13.0, 13.0))
vector((-3.32, 7.13, 0.78), (6.44, 6.44, 6.44))
vector((-5.77, 6.43, -2.35), (-1.96, -1.96, -1.96))
vector((-8.92, 2.51, -2.88), (-11.75, -11.75, -11.75))
vector((4.5, -5.61, -5.3), (-9.52, -9.52, -9.52))
vector((2.05, 0.26, -4.64), (-4.4, -4.4, -4.4))
vector((-0.66, -3.18, -7.23), (-28.2, -28.2, -28.2))
vector((-8.89, -3.36, -1.04), (-25.18, -25.18, -25.18))
vector((-5.59, 2.67, -1.97), (-11.83, -11.83, -11.83))
vector((5.53, 7.25, 2.82), (55.36, 55.36, 55.36))
vector((-8.13, 8.41, 8.63), (5.29, 5.29, 5.29))
vector((-5.4, 6.3, -0.58), (-0.11, -0.11, -0.11))
vector((-8.6, -0.89, -3.33), (-25.86, -25.86, -25.86))
vector((4.05, 4.51, -9.34), (0.12, 0.12, 0.12))
vector((-6.03, -0.8, 5.97), (-1.31, -1.31, -1.31))
vector((5.31, 4.9, -7.15), 

In [ ]:
repulsion_model(pole, np.array)

array([9.43672038, 8.45536316, 7.90586395])